### 1. 최소신장트리의 이해
1. 신장트리(Spanning Tree)란?
    - 원래의 그래프의 모든 노드가 연결되어 있으면서, 트리의 속성을 만족하는 그래프
    - 신장트리의 조건
        - 본래의 그래프의 모든 노드를 포함해야함
        - 모든 노드가 서로 연결
        - 트리의 속성을 만족(사이클 존재 X)
    
2. 최소신장트리
    - Minimum Spanning Tree(MST)라고 불리움
    - 가능한 Spanning Tree 중, 간선의 가중치 합이 최소인 Spanning Tree 를 지칭함
    
3. 최소신장트리 알고리즘
    - 그래프에서 최소 신장 트리를 찾을 수 있는 알고리즘 존재
    - 대표적인 신장트리 알고리즘
        - 크루스칼 알고리즘
        - 프림알고리즘

### 2. 크루스칼 알고리즘
1. 모든 정점을 독립적인 집합으로 만든다.
2. 모든 간선 비용을 기준으로 정렬, 비용이 작은 간선부터 양 끝의 두 정점을 비교한다. (union- find)
    - 집합 안에, 양 끝 두 정점이 들어있다면, 사이클이 생기니까 해당 간선은 포함시키지 않는다.
3. 두 정점의 최상위 정점을 확인하고, 서로 다를 경우 두 정점을 연결한다.(union- find)
    - 사이클이 생기지 않도록하는 것 
> 탐욕 알고리즘을 기초로 하고있음(당장 눈앞의 최소 비용을 선택하여, 결과적으로 최적의 솔루션을 찾음)

### 3. Union-Find 알고리즘
- Disjoint Set을 표현할때 사용하는 알고리즘으로, 트리구조 활용하는 알고리즘
- 간단하게, 노드들 중에 연결된 노드를 찾거나, 노드들을 서로 연결할때 (합칠 때) 사용
- Disjoint Set이란
    - **서로 중복되지 않는 부분 집합들로 나눠진 원소들에 대한 정보를 저장하고 조작하는 자료구조**
    - 공통 원소가 없는(서로소) 상호 배타적인 부분 집합들로 나눠진 원소들에 대한 자료구조 의미
    - **Disjoint Set == 서로소 집합 자료구조**
- 구현
    1. 초기화: n개의 원소가 개별 집합으로 이루어지도록 초기화
    2. union: 두 개별 집합을 하나의 집합으로 합침, 두 트리를 하나의 트리로 만듬
    3. find: 여러 노드가 존재할 때, 두 개의 노드를 선택해서, 현재 두 노드가 서로 같은 그래프에 속하는지 판별하기 위해,  
        각 그룹의 최 상단원소(즉, 루트노드를) 확인
- 고려할 점
    - union 순서에 따라, 최악의 경우 링크드 리스트와 같은 형태가 될 수 있음. O(N)
    - 해당 문제를 위해, union-by-rank, path compression 기법을 사용
        - union-by-rank 기법
            - 각 트리에 대한 높이(rank)를 기억해두고
            - Union시, 두 트리 높이(rank)가 다르면, 높이가 작은 트리를 높이가 큰 트리에 붙임.  
                (즉, 높이가 큰 트리의 루트노드가 합친 집합의 루트노드가 되게함)
            - 두 트리 높이가 같으면, 한 쪽 트리 높이를 1 증가시킨 후, 다른쪽 트리를 해당 트리에 붙임
            - union-by-rank 기법을 활용하면, O(logN)의 시간 복잡도를 가짐.
        - Path compression
            - Find를 실행한 노드에서 거쳐간 노드를 루트에 다이렉트로 연결하는 기법
            - Find를 실행한 노드 이후부터는 루트 노드를 한번에 알 수 있음
        - union-by-rank와 path compression 기법을 사용하면, 시간복잡도 O(MlogM)
            - O(logN)은 최악의 경우, O(1)에 가까움

### 4. 크루스칼 알고리즘 구현

In [5]:
mygraph ={
    'vertices':['A','B','C','D','E','F','G'],
    'edges':[
        (7,'A','B'),
        (5,'A','D'),
        (7,'B','A'),
        (9,'B','D'),
        (8,'B','C'),
        (7,'B','E'),
        (8,'C','B'),
        (5,'C','E'),
        (5,'D','A'),
        (9,'D','B'),
        (7,'D','E'),
        (6,'D','F'),
        (5,'E','C'),
        (7,'E','D'),
        (8,'E','F'),
        (9,'E','G'),
        (6,'F','D'),
        (8,'F','E'),
        (11,'F','G'),
        (9,'G','E'),
        (11,'G','F')
    ]
}

In [6]:
parent = dict() # 각 노드마다, 부모노드의 값을 저장
rank = dict() # 각 노드의 rank 번호(높이)

def find(node):
    # path compression
    if parent[node] != node: # (부모노드와 해당노드가 다르다면) 루트 노드가 있다는 의미 
        parent[node] = find(parent[node]) # 해당노드의 부모노드를 재귀적으로 계속 찾아감.
    return parent[node] #r결과적으로 루트노드를 반환
        

def union(node_v, node_u):
    root1 = find(node_v)
    root2 = find(node_u)
    
    # union-by-rank 기법
    if rank[root1] > rank [root2]:
        parent[root2] = root1
    else:
        parent[root1] = root2
        if rank[root1] == rank[root2]:
            rank[root2] += 1

def make_set(node):
    parent[node] = node
    rank[node] = 0

def kruskal(graph):
    mst = list()
    
    #1. 초기화
    for node in graph['vertices']:
        make_set(node)    
        
    # 2. 간선의 가중치 순으로 정렬
    edges = graph['edges']
    edges.sort()
    
    # 3. 간선연결 (사이클 없는)
    # 각 루트노드 같은지 여부 파악 다르다면 union
    for edge in edges:
        weigh, node_v, node_u = edge
        if find(node_v) != find(node_u):
            union(node_v, node_u)
            mst.append(edge)
    
    return mst

In [7]:
kruskal(mygraph)

[(5, 'A', 'D'),
 (5, 'C', 'E'),
 (6, 'D', 'F'),
 (7, 'A', 'B'),
 (7, 'B', 'E'),
 (9, 'E', 'G')]

### 5. 시간복잡도
- O(ElogE)
    - 초기화 하는 과정은 노드의 수만큼 O(V)
    - 간선을 기준으로 sort() => O(ElogE)
    - Union- Find => O(E) 
- 가장 오래 걸리는 시간 O(ElogE)